In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=7)

In [ ]:
train_high = h2o.import_file('../cleaned-data/04-train_high-catAsInt-withXYZ.csv')
train_low = h2o.import_file('../cleaned-data/04-train_low-catAsInt-withXYZ.csv')
test_high = h2o.import_file('../cleaned-data/04-test_high-catAsInt-withXYZ.csv')
test_low = h2o.import_file('../cleaned-data/04-test_low-catAsInt-withXYZ.csv')      
train_high.head()

In [ ]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

In [ ]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

In [ ]:
train_frame = train_high
model_name = 'diamonds_high'
runtime = 3540

aml_high = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

In [ ]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [ ]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

In [ ]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [ ]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

In [ ]:
h2o.export_file(frame=combinedTable,path='../predictions/16-Auto-ML-Split-data-catAsInt-withXYZ.csv')